In [ ]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

TABELA_ORIGEM = "spotify_analytics.silver.tb_tracks"
TABELA_DESTINO = "spotify_analytics.gold.dm_tempo"

### Configuração

Define origem (Silver tb_tracks) e destino (Gold dm_tempo).

### Transformação Silver → Gold (Dimensão Tempo)

Cria dimensão temporal com todas as datas de lançamento presentes no dataset.

**Processo:**
1. Extrai datas únicas de release_date
2. Para cada data, deriva:
   - cd_tempo: Formato YYYYMMDD (ex: 20240515)
   - Componentes: ano, mês, dia
   - Nome do mês por extenso
   - Trimestre (Q1, Q2, Q3, Q4)
   - Semestre (S1, S2)
   - Década (2020s, 2010s, etc.)

**Decisão Arquitetural**: Dimensão tempo pré-populada vs on-demand:
- **Abordagem escolhida**: On-demand (apenas datas existentes nos dados)
- **Alternativa**: Pre-populate (gerar 50 anos de datas antecipadamente)
- **Justificativa**: Dataset limitado, não precisamos de datas futuras
- **Trade-off**: Se precisar analisar períodos sem lançamentos, não terão linhas na dimensão

Faz MERGE em dm_tempo (Gold).

In [ ]:
df_silver = spark.read.table(TABELA_ORIGEM)

# Extrai datas únicas
df_dates = (
    df_silver
    .select("release_date")
    .filter(F.col("release_date").isNotNull())
    .distinct()
)

# Deriva atributos temporais
df_gold = df_dates.select(
    # Surrogate key: YYYYMMDD
    F.date_format(F.col("release_date"), "yyyyMMdd").cast("int").alias("cd_tempo"),
    
    F.col("release_date").alias("dt_completa"),
    
    # Componentes numéricos
    F.year("release_date").alias("nu_ano"),
    F.month("release_date").cast("smallint").alias("nu_mes"),
    F.dayofmonth("release_date").cast("smallint").alias("nu_dia"),
    
    # Nome do mês
    F.date_format("release_date", "MMMM").alias("nm_mes"),
    
    # Trimestre
    F.concat(F.lit("Q"), F.quarter("release_date")).alias("ds_trimestre"),
    
    # Semestre
    F.when(F.month("release_date") <= 6, "S1").otherwise("S2").alias("ds_semestre"),
    
    # Década
    F.concat(
        (F.floor(F.year("release_date") / 10) * 10).cast("string"),
        F.lit("s")
    ).alias("ds_decada")
)

print(f"Total de datas únicas: {df_gold.count()}")
df_gold.orderBy(F.desc("cd_tempo")).show(10, truncate=False)

### MERGE em dm_tempo

Match por cd_tempo (surrogate key YYYYMMDD).

In [ ]:
DeltaTable.forName(spark, TABELA_DESTINO).alias("t").merge(
    df_gold.alias("s"),
    "t.cd_tempo = s.cd_tempo"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Dimensão Tempo carregada!")

### Verificação Final

In [ ]:
# Distribuição por década
spark.sql(f"""
    SELECT ds_decada, COUNT(*) as qtd_datas
    FROM {TABELA_DESTINO}
    GROUP BY ds_decada
    ORDER BY ds_decada DESC
""").show()

# Distribuição por trimestre (últimos 2 anos)
spark.sql(f"""
    SELECT nu_ano, ds_trimestre, COUNT(*) as qtd_datas
    FROM {TABELA_DESTINO}
    WHERE nu_ano >= (SELECT MAX(nu_ano) - 1 FROM {TABELA_DESTINO})
    GROUP BY nu_ano, ds_trimestre
    ORDER BY nu_ano DESC, ds_trimestre
""").show()